# NYC Street Trees Mapmaking

The following is meant more as a sandbox than a polished workbook, for making various maps. Be aware that the html maps get quite large. Enjoy :)

## Cleaning and exploring the data
* Import the necessary libraries.

In [2]:
# dataframe libraries
import pandas as pd
import numpy as np
import geopandas as gpd

# visualizations libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set_style('ticks')
import folium

# custom functions, reloading when updated
from functions import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


* Import the data and take a look.

In [11]:
# load the clean dataframe
with gzip.open('data/nyc_trees_initial_clean.pkl', 'rb') as hello:
    trees_mapmaking = pickle.load(hello)
    
# take a look at the first five rows
trees_mapmaking.head()

## Mapmaking

In [48]:
# Make a list of latitude, longitude, and health status for all datapoints
latlon = [(lat, lon, health) for lat, lon, health in zip(list(trees_mapmaking.latitude),
                                                         list(trees_mapmaking.longitude),
                                                         list(trees_mapmaking.health))]
latlon[:5]

[(40.72309177, -73.84421522, 'Fair'),
 (40.79411067, -73.81867946, 'Fair'),
 (40.71758074, -73.93660770000002, 'Good'),
 (40.71353749, -73.93445616, 'Good'),
 (40.66677776, -73.97597938, 'Good')]

* Uncomment code below to make a map of all the datapoints. *WARNING: takes awhile and the output html file is over 300MB and very laggy.*

In [49]:
# tree_map = folium.Map(location=[40.700991, -73.924587], zoom_start=11)

# for coord in latlon:
#     if coord[2] == 'Good':
#         folium.Circle(location=[coord[0], coord[1]], radius=1, color='green').add_to(tree_map)
#     elif coord[2] == 'Fair':
#         folium.Circle(location=[coord[0], coord[1]], radius=1, color='yellow').add_to(tree_map)
#     else:
#         folium.Circle(location=[coord[0], coord[1]], radius=1, color='red').add_to(tree_map)

# tree_map.save('maps/tree_health_map.html')

* Since the above maps are impractical to use, I'll make some maps on a smaller scale.
    * First let's look at the five community board areas with the highest proportion of trees in 'Good' health.
    * Then let's look at the five community board areas with the highest proportion of trees in 'Poor' health.
    
### Healthy trees map

In [43]:
healthiest = trees_mapmaking.groupby(['nta']).health.value_counts(normalize=True).unstack().sort_values('Good', ascending=False).head(10)
healthiest.index


Index(['MN50', 'QN02', 'QN49', 'MN13', 'BK93', 'SI07', 'SI28', 'BX37', 'SI35',
       'QN07'],
      dtype='object', name='nta')

In [44]:
# create new dataframe with just these ten neighborhoods
most_good_trees = trees_mapmaking[trees_mapmaking.nta.isin(list(healthiest.index))]
# find the average coordinates, so we can set the center of our map
most_good_trees[['latitude', 'longitude']].mean()

latitude     40.708752
longitude   -73.945031
dtype: float64

In [45]:
# Make a list of latitude, longitude, and health status for all datapoints
latlon_best = [(lat, lon, health) for lat, lon, health in zip(list(most_good_trees.latitude),
                                                         list(most_good_trees.longitude),
                                                         list(most_good_trees.health))]
latlon_best[:5]

[(40.79411067, -73.81867946, 'Fair'),
 (40.7337165, -73.97705764, 'Fair'),
 (40.793138, -73.81946649, 'Good'),
 (40.73357762, -73.97672526, 'Fair'),
 (40.73346807, -73.97646308, 'Good')]

In [54]:
healthy_nta_map = folium.Map(location=[40.708752, -73.945031], zoom_start=11)

for coord in latlon_best:
    if coord[2] == 'Good':
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='green').add_to(healthy_nta_map)
    elif coord[2] == 'Fair':
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='yellow').add_to(healthy_nta_map)
    else:
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='red').add_to(healthy_nta_map)

healthy_nta_map.save('maps/healthy_nta_map.html')

### Poorest health trees map

In [67]:
poorest_health = trees_mapmaking.groupby(['nta']).health.value_counts(normalize=True).unstack().sort_values('Poor', ascending=False).head(10)
poorest_health.index

Index(['QN10', 'QN12', 'MN31', 'MN32', 'MN36', 'QN20', 'MN20', 'MN17', 'MN01',
       'MN34'],
      dtype='object', name='nta')

In [50]:
poorest_health = trees_mapmaking.groupby(['nta']).health.value_counts(normalize=True).unstack().sort_values('Good', ascending=True).head(10)
poorest_health.index


Index(['QN12', 'QN10', 'MN17', 'MN31', 'BK23', 'BK21', 'MN20', 'MN35', 'MN01',
       'BX01'],
      dtype='object', name='nta')

In [51]:
# create new dataframe with just these five community boards
most_poor_trees = trees_mapmaking[trees_mapmaking.nta.isin(list(poorest_health.index))]
# find the average coordinates, so we can set the center of our map
most_poor_trees[['latitude', 'longitude']].mean()

latitude     40.710840
longitude   -73.904969
dtype: float64

In [52]:
# Make a list of latitude, longitude, and health status for all datapoints
latlon_worst = [(lat, lon, health) for lat, lon, health in zip(list(most_poor_trees.latitude),
                                                         list(most_poor_trees.longitude),
                                                         list(most_poor_trees.health))]
latlon_worst[:5]

[(40.74503399, -73.98253015, 'Fair'),
 (40.74829709, -73.98065645, 'Good'),
 (40.59626688, -73.77234286, 'Fair'),
 (40.59683648, -73.77245394, 'Poor'),
 (40.77277225, -73.95532709999998, 'Good')]

In [55]:
poor_health_nta_map = folium.Map(location=[40.710840, -73.904969], zoom_start=11)

for coord in latlon_worst:
    if coord[2] == 'Good':
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='green').add_to(poor_health_nta_map)
    elif coord[2] == 'Fair':
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='yellow').add_to(poor_health_nta_map)
    else:
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='red').add_to(poor_health_nta_map)

poor_health_nta_map.save('maps/poor_health_nta_map.html')

### Bushwick trees
- Make a map on a smaller scale, using my neighborhood of Bushwick as an example.
- *NOTE: you can use any community board number or combination of neighborhoods.*

In [57]:
bushwick = trees_mapmaking[trees_mapmaking.cb_num == 304]
# find the average coordinates, so we can set the center of our map
bushwick[['latitude', 'longitude']].mean()

latitude     40.696207
longitude   -73.918556
dtype: float64

In [58]:
latlon_bushwick = [(lat, lon, health) for lat, lon, health in zip(list(bushwick.latitude),
                                                                  list(bushwick.longitude),
                                                                  list(bushwick.health))]
latlon_bushwick[:5]

[(40.69775112, -73.90933909, 'Fair'),
 (40.70233782, -73.91521777, 'Good'),
 (40.69791922, -73.90830664, 'Good'),
 (40.68280728, -73.90979496, 'Good'),
 (40.70256662, -73.91498455, 'Good')]

In [59]:
bushwick_map = folium.Map(location=[40.696207, -73.918556], zoom_start=15)

for coord in latlon_bushwick:
    if coord[2] == 'Good':
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='green').add_to(bushwick_map)
    elif coord[2] == 'Fair':
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='yellow').add_to(bushwick_map)
    else:
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='red').add_to(bushwick_map)

bushwick_map.save('maps/bushwick_tree_health_map.html')

### Maps by census taker (professional vs. volunteer)

In [63]:
# create new dataframe with just professional census takers
pro_trees = trees_mapmaking[trees_mapmaking.user_type.isin(['TreesCount Staff', 'NYC Parks Staff'])]
# create new dataframe with just volunteer census takers
vol_trees = trees_mapmaking[trees_mapmaking.user_type == 'Volunteer']

In [64]:
# Make a list of latitude, longitude, and health status for all datapoints
latlon_pros = [(lat, lon, health) for lat, lon, health in zip(list(pro_trees.latitude),
                                                         list(pro_trees.longitude),
                                                         list(pro_trees.health))]

latlon_vols = [(lat, lon, health) for lat, lon, health in zip(list(vol_trees.latitude),
                                                         list(vol_trees.longitude),
                                                         list(vol_trees.health))]

* Uncomment code below to make a map of all the datapoints. *WARNING: takes awhile and the output html file is over 300MB and very laggy.*

In [65]:
# pro_map = folium.Map(location=[40.700991, -73.924587], zoom_start=11)

# for coord in latlon_pros:
#     if coord[2] == 'Good':
#         folium.Circle(location=[coord[0], coord[1]], radius=1, color='green').add_to(pro_map)
#     elif coord[2] == 'Fair':
#         folium.Circle(location=[coord[0], coord[1]], radius=1, color='yellow').add_to(pro_map)
#     else:
#         folium.Circle(location=[coord[0], coord[1]], radius=1, color='red').add_to(pro_map)

# pro_map.save('maps/pro_census_map.html')

In [66]:
# vol_map = folium.Map(location=[40.700991, -73.924587], zoom_start=11)

# for coord in latlon_vols:
#     if coord[2] == 'Good':
#         folium.Circle(location=[coord[0], coord[1]], radius=1, color='green').add_to(vol_map)
#     elif coord[2] == 'Fair':
#         folium.Circle(location=[coord[0], coord[1]], radius=1, color='yellow').add_to(vol_map)
#     else:
#         folium.Circle(location=[coord[0], coord[1]], radius=1, color='red').add_to(vol_map)

# vol_map.save('maps/vol_census_map.html')

### Best/worst community board

In [78]:
trees_mapmaking.groupby(['cb_num']).health.value_counts(normalize=True).unstack().sort_values('Good', ascending=False).head(1)

health,Fair,Good,Poor
cb_num,,,
210,0.095958,0.877063,0.026979


In [77]:
trees_mapmaking.groupby(['cb_num']).health.value_counts(normalize=True).unstack().sort_values('Poor', ascending=False).head(1)


health,Fair,Good,Poor
cb_num,,,
414,0.254523,0.64509,0.100386


In [80]:
best_cb = trees_mapmaking[trees_mapmaking.cb_num == 210]
# find the average coordinates, so we can set the center of our map
best_cb[['latitude', 'longitude']].mean()

latitude     40.839529
longitude   -73.823598
dtype: float64

In [79]:
worst_cb = trees_mapmaking[trees_mapmaking.cb_num == 414]
# find the average coordinates, so we can set the center of our map
worst_cb[['latitude', 'longitude']].mean()

latitude     40.593158
longitude   -73.794398
dtype: float64

In [81]:
# Make a list of latitude, longitude, and health status for all datapoints
latlon_best_cb = [(lat, lon, health) for lat, lon, health in zip(list(best_cb.latitude),
                                                         list(best_cb.longitude),
                                                         list(best_cb.health))]

latlon_worst_cb = [(lat, lon, health) for lat, lon, health in zip(list(worst_cb.latitude),
                                                         list(worst_cb.longitude),
                                                         list(worst_cb.health))]

In [82]:
best_cb_map = folium.Map(location=[40.839529, -73.823598], zoom_start=15)

for coord in latlon_best_cb:
    if coord[2] == 'Good':
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='green').add_to(best_cb_map)
    elif coord[2] == 'Fair':
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='yellow').add_to(best_cb_map)
    else:
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='red').add_to(best_cb_map)

best_cb_map.save('maps/best_cb_map.html')

In [83]:
worst_cb_map = folium.Map(location=[40.593158, -73.794398], zoom_start=15)

for coord in latlon_worst_cb:
    if coord[2] == 'Good':
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='green').add_to(worst_cb_map)
    elif coord[2] == 'Fair':
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='yellow').add_to(worst_cb_map)
    else:
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='red').add_to(worst_cb_map)

worst_cb_map.save('maps/worst_cb_map.html')